In [2]:
import mackinac
import cobra
import pandas as pd
import json
import os
import numpy as np

In [3]:
# load ID's for each organisms genome
id_table = pd.read_table('../data/study_strain_subset.tsv',sep='\t',dtype=str)
id_table = id_table.replace(np.nan, '', regex=True)
species_to_id = dict(zip(id_table["designation in screen"],id_table["PATRIC genome ID"]))

In [4]:
species_to_id

{'B. clarus': '762984.10',
 'B. longum subsp. longum': '',
 'B. vulgatus HM-720': '997891.3',
 'C. saccharolyticum': '610130.3',
 'L. lactis': '272623.7',
 'L. plantarum': '220668.9',
 'P. distasonis': '435591.13',
 'P. merdae': '411477.4',
 'R. gnavus': '411470.41'}

In [5]:
mackinac.get_token('gregmedlock_seed')


patric password:  ···············


'gregmedlock_seed@patricbrc.org'

In [ ]:
# grab and save a universal model to be used later for gapfilling. This is a public template available in Mike Mundy's workspace.
# The template says "gramneg", but there is no difference between the g+ and g- templates other than biomass composition,
# which will not be used during gapfilling (the GENREs will already have their own biomass function).
gramneg = mackinac.create_universal_model('/mmundy/public/modelsupport/templates/MicrobialNegativeResolved.modeltemplate')
cobra.io.save_json_model(gramneg,'../data/universal_mundy.json')

In [7]:
# save id's and both names in dictionary
name_to_recon_info = {}
name_to_gapfill_solution = {}
for species in species_to_id.keys():
    # Check for an existing GENRE and make sure there is a PATRIC ID for the strain--
    # if there is no PATRIC ID, the dictionary will have an empty string for that strain.
    if species+'.json' not in os.listdir('../data/modelseed_models') and species_to_id[species]:
        species_id = species_to_id[species]

        # reconstruct model; function returns a dictionary with reconstruction info, NOT the model
        print("Reconstructing GENRE for " + species)
        recon_info = mackinac.create_patric_model(species_id,species)
        name_to_recon_info[species] = recon_info
        # Get the reactions contained in the gapfill solution. This is on complete media
        name_to_gapfill_solution[species] = mackinac.get_patric_gapfill_solutions(species)[0]
        # convert to a cobra model
        model = mackinac.create_cobra_model_from_patric_model(species)
        # Save model in json format using patric_id
        cobra.io.save_json_model(model, '../data/modelseed_models/'+species+'.json')

# save conversion dict for id:original_name:SEED_name mapping
with open('../data/patric_recon_info.json','w') as jsonfile:
    json.dump(name_to_recon_info,jsonfile)
    
# save the gapfill solutions
with open('../data/patric_gapfill_solutions.json','w') as jsonfile:
    json.dump(name_to_gapfill_solution,jsonfile)
    

Reconstructing GENRE for P. distasonis


KeyboardInterrupt: 